In [16]:
from typing import Any, Tuple
import json
import hashlib
from filelock import FileLock


def set_unsafe(primary_key : str, content : Any):
    # write etag and content
    serialized_content = json.dumps(content)
    with open(_get_path(primary_key), 'w') as fh:
        fh.write(serialized_content)


def get(primary_key : str) -> Tuple[Any, str]:
    try:
        with open(_get_etag(primary_key), 'r') as fh:
            etag = fh.read()
    except FileDoesNotExist:
        etag = ''

    with open(_get_path(primary_key), 'r') as fh:
        content = json.loads(fh.read())
    return content, etag


import os
storage_base = os.path.join(os.curdir, "_filestore")
os.makedirs(storage_base, exist_ok=True)

def _get_path(key : str):
    return os.path.join(storage_base, key)

def _get_etag(key : str):
    return os.path.join(storage_base, f'{key}_etag')

def _get_lock(key : str):
    return os.path.join(storage_base, f'{key}_.lock')

In [18]:
from typing import Any, Tuple
import json
import hashlib
from filelock import FileLock



# NOTE! This doesn't account for multi-threaded access.
def set_safe(primary_key : str, content : Any, etag : str = 'required but not set'):
    if os.path.exists(_get_etag(primary_key)):
        stored_etag = open(_get_etag(primary_key), 'r').read()  # may throw file errors.
        if stored_etag != etag:
            raise ValueError(f"Invalid etag! Someone moved your cheese?\nOld: {stored_etag}\nNew: {etag}")
    # write etag and content
    serialized_content = json.dumps(content)
    new_etag = hashlib.md5(serialized_content.encode('ascii', 'xmlcharrefreplace'))
    with open(_get_etag(primary_key), 'w') as fh:
        fh.write(str(new_etag.hexdigest()))
    with open(_get_path(primary_key), 'w') as fh:
        fh.write(serialized_content)



In [22]:

def set_safe(primary_key : str, content : Any, etag : str = 'required but not set'):
    locker = FileLock(_get_lock(primary_key), timeout=1)
    with locker:
        if os.path.exists(_get_etag(primary_key)):
            stored_etag = open(_get_etag(primary_key), 'r').read()  # may throw file errors.
            if stored_etag != etag:
                raise ValueError(f"Invalid etag! Someone moved your cheese?\nOld: {stored_etag}\nNew: {etag}")
        # write etag and content
        serialized_content = json.dumps(content)
        new_etag = hashlib.md5(serialized_content.encode('ascii', 'xmlcharrefreplace'))

        with open(_get_etag(primary_key), 'w') as fh:
            fh.write(str(new_etag.hexdigest()))
        with open(_get_path(primary_key), 'w') as fh:
            fh.write(serialized_content)




In [19]:
base_content = {
    'paragraphs': {
        0: {
            'text': "Paragaph 1",
            'author': 'A'
        }
    },
    'last_modified_by': 'A'
}
set_unsafe("blogpost", base_content)
get('blogpost')

({'paragraphs': {'0': {'text': 'Paragaph 1', 'author': 'A'}},
  'last_modified_by': 'A'},
 '54a706a96bc31ef5e54b07606d6033cd')

Show a conflict

In [12]:
# Unsafe get/set

# Agent A
a_content, _ = get('blogpost')

# Agent B
b_content, _ = get('blogpost')

# Updates
a_content['last_modified_by'] = 'A'
a_content['paragraphs'][0] = {'text': 'concurrency is hard', 'author': "A"}

b_content['last_modified_by'] = 'B'
b_content['paragraphs'][1] = {'text': 'concurrency is easy!', 'author': "B"}

# Write
set_unsafe('blogpost', b_content)


In [13]:
get('blogpost')

({'paragraphs': {'0': {'text': 'Paragaph 1', 'author': 'A'},
   '1': {'text': 'concurrency is easy!', 'author': 'B'}},
  'last_modified_by': 'B'},
 '54a706a96bc31ef5e54b07606d6033cd')

In [14]:
set_unsafe('blogpost', a_content)

In [15]:
get('blogpost')

({'paragraphs': {'0': {'text': 'concurrency is hard', 'author': 'A'}},
  'last_modified_by': 'A'},
 '54a706a96bc31ef5e54b07606d6033cd')

In [21]:
set_unsafe("safeblogpost", base_content)  # for later

In [23]:
# Agent A
a_content, etag = get('safeblogpost')

# Agent B
b_content, b_etag = get('safeblogpost')

# Updates
a_content['last_modified_by'] = 'A'
a_content['paragraphs'][0] = {'text': 'concurrency is hard', 'author': "A"}

b_content['last_modified_by'] = 'B'
b_content['paragraphs'][1] = {'text': 'concurrency is easy!', 'author': "B"}

# Write
set_safe('safeblogpost', b_content, b_etag)

NameError: name 'FileDoesNotExist' is not defined